In [1]:
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.legacy.embeddings import HuggingFaceEmbedding
from llama_index.legacy.llms import (CustomLLM, CompletionResponse, CompletionResponseGen, LLMMetadata)
from llama_index.legacy.embeddings import HuggingFaceEmbedding

from typing import Any

In [2]:
emb_path = "../../../Model/moka-ai/m3e-base"
embedding_model = HuggingFaceEmbedding(emb_path)

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pathlib import Path
from typing import Union

from peft import AutoPeftModelForCausalLM, PeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    PreTrainedTokenizerFast,
)

ModelType = Union[PreTrainedModel, PeftModelForCausalLM]
TokenizerType = Union[PreTrainedTokenizer, PreTrainedTokenizerFast]

def _resolve_path(path: Union[str, Path]) -> Path:
    return Path(path).expanduser().resolve()


def load_model_and_tokenizer(model_dir: Union[str, Path]) -> tuple[ModelType, TokenizerType]:
    model_dir = _resolve_path(model_dir)
    if (model_dir / 'adapter_config.json').exists():
        model = AutoPeftModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=True, device_map='auto'
        )
        tokenizer_dir = model.peft_config['default'].base_model_name_or_path
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=True, device_map='auto'
        )
        tokenizer_dir = model_dir
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_dir, trust_remote_code=True
    )
    return model, tokenizer

In [4]:
# model_dir = "output/checkpoint-1000"
model_dir = "../../../Model/THUDM/chatglm3-6b"

model, tokenizer = load_model_and_tokenizer(model_dir)

Loading checkpoint shards: 100%|██████████| 7/7 [03:05<00:00, 26.55s/it]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [5]:
# set context window size
context_window = 2048
# set number of output tokens
num_output = 512

model_name = "chatglm3-6b"

class ChatGML(CustomLLM):
    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            context_window=context_window,
            num_output=num_output,
            model_name=model_name,
        )

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        # prompt_length = len(prompt)

        # only return newly generated tokens
        text,_ = model.chat(tokenizer, prompt, history=[])
        return CompletionResponse(text=text)

    @llm_completion_callback()
    def stream_complete(
        self, prompt: str, **kwargs: Any
    ) -> CompletionResponseGen:
        raise NotImplementedError()

llm_model = ChatGML()

In [6]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader('embedding_data').load_data()
service_context = ServiceContext.from_defaults(llm=llm_model, embed_model=embedding_model)

/tmp/ipykernel_131078/3054200758.py:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm_model, embed_model=embedding_model)


In [7]:
# create index
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [23]:
# query engine
query_engine = index.as_query_engine()

# query
# question = "你现在是一个金融领域专家，你需要根据用户的query编排api，需要使用的api在下面给出。query是：我想了解汇丰晋信2016C的基金经理是谁，目前的基金规模以及申购费率是多少? \nquery中使用到的api可能是：条件选基_查询代码、条件选基_查询类型、条件选基_查询规模、条件选基_查询晨星评级、条件选基_查询蚂蚁评级、条件选基_查询申购费率、条件选基_查询成立年限、条件选基_查询开放类型、条件选基_查询申购状态、条件选基_查询蚂蚁金选标识、条件选基_查询风险等级、条件选基_查询基金经理类型、条件选基_查询基金经理、条件选基_查询基金公司、条件选基_查询基金经理从业年限、条件选基_查询基金经理管理规模、条件选基_查询基金经理年化回报率、条件选基_查询持股估值、条件选基_查询持股集中度、条件选基_查询持股热门度、条件选基_查询持股换手率、条件选基_查询机构占比、条件选基_查询单一持有人占比、条件选基_查询重仓股票、条件选基_查询重仓行业、条件选基_查询持仓风格、条件选基_查询基金公司是否自购、条件选基_查询是否可投沪港深、条件选基_查询近一年买入持有6个月历史盈利概率、条件选基_查询近3年买入持有1年历史盈利概率、条件选基_查询近5年买入持有1年历史盈利概率、条件选基_查询近期收益率、条件选基_查询近期收益率同类排名、条件选基_查询近期年化收益率、条件选基_查询近期年化收益率同类排名、条件选基_查询近期超大盘收益率、条件选基_查询近期创新高次数、条件选基_查询近期最大回撤、条件选基_查询近期最大回撤同类排名、条件选基_查询近期波动率同类排名、条件选基_查询近期最长解套天数、条件选基_查询近期跟踪误差、条件选基_查询近期夏普比率同类排名、条件选基_查询近期卡玛比率同类排名、条件选基_查询近期信息比率同类排名、条件选基_查询年度收益率、条件选基_查询近期指数增强、条件选基_查询年度最大回撤、条件选基_查询基金事后分类、条件选基_查询基金资产打分、条件选基_查询基金募集类型、条件选基_查询投组区域、条件选基_查询基金份额类型(A、B、C)、条件选基_查询申购最小起购金额、条件选基_查询分红方式、条件选基_查询赎回状态、条件选基_查询是否支持定投、条件选基_查询单位净值同步日期、条件选基_查询单位净值、条件选基_查询基金经理任职期限、条件选基_查询近期夏普率、条件选基_查询投资市场、条件选基_查询板块、条件选基_查询行业、条件选基_查询聚类行业、条件选基_查询管理费率、条件选基_查询销售服务费率、条件选基_查询托管费率、条件选基_查询认购费率、条件选基_查询赎回费率、条件选基_查询分红年度、条件选基_查询权益登记日、条件选基_查询除息日、条件选基_查询派息日、条件选基_查询红利再投日、条件选基_查询每十份收益单位派息（元）、基金查询_查询代码、基金查询_查询类型、基金查询_查询规模、基金查询_查询晨星评级、基金查询_查询蚂蚁评级、基金查询_查询申购费率、基金查询_查询成立年限、基金查询_查询开放类型、基金查询_查询申购状态、基金查询_查询蚂蚁金选标识、基金查询_查询风险等级、基金查询_查询基金经理类型、基金查询_查询基金经理、基金查询_查询基金公司、基金查询_查询基金经理从业年限、基金查询_查询基金经理管理规模、基金查询_查询基金经理年化回报率、基金查询_查询持股估值、基金查询_查询持股集中度、基金查询_查询持股热门度、基金查询_查询持股换手率、基金查询_查询机构占比、基金查询_查询单一持有人占比、基金查询_查询重仓股票、基金查询_查询重仓行业、基金查询_查询持仓风格、基金查询_查询基金公司是否自购、基金查询_查询是否可投沪港深、基金查询_查询近一年买入持有6个月历史盈利概率、基金查询_查询近3年买入持有1年历史盈利概率、基金查询_查询近5年买入持有1年历史盈利概率、基金查询_查询近期收益率、基金查询_查询近期收益率同类排名、基金查询_查询近期年化收益率、基金查询_查询近期年化收益率同类排名、基金查询_查询近期超大盘收益率、基金查询_查询近期创新高次数、基金查询_查询近期最大回撤、基金查询_查询近期最大回撤同类排名、基金查询_查询近期波动率同类排名、基金查询_查询近期最长解套天数、基金查询_查询近期跟踪误差、基金查询_查询近期夏普比率同类排名、基金查询_查询近期卡玛比率同类排名、基金查询_查询近期信息比率同类排名、基金查询_查询年度收益率、基金查询_查询近期指数增强、基金查询_查询年度最大回撤、基金查询_查询基金事后分类、基金查询_查询基金资产打分、基金查询_查询基金募集类型、基金查询_查询投组区域、基金查询_查询基金份额类型、基金查询_查询申购最小起购金额、基金查询_查询分红方式、基金查询_查询赎回状态、基金查询_查询是否支持定投、基金查询_查询单位净值同步日期、基金查询_查询单位净值、基金查询_查询基金经理任职期限、基金查询_查询近期夏普率、基金查询_查询投资市场、基金查询_查询板块、基金查询_查询行业、基金查询_查询聚类行业、基金查询_查询管理费率、基金查询_查询销售服务费率、基金查询_查询托管费率、基金查询_查询认购费率、基金查询_查询赎回费率、基金查询_查询分红年度、基金查询_查询权益登记日、基金查询_查询除息日、基金查询_查询派息日、基金查询_查询红利再投日、基金查询_查询每十份收益单位派息（元）、条件选股_查询代码、条件选股_查询开盘价、条件选股_查询最高价、条件选股_查询最低价、条件选股_查询当前价、条件选股_查询收盘价、条件选股_查询成交量、条件选股_查询成交额、条件选股_查询涨停价、条件选股_查询跌停价、条件选股_查询涨跌额、条件选股_查询涨跌幅、条件选股_查询换手率、条件选股_查询每股收益、条件选股_查询静态市盈率、条件选股_查询总市值、条件选股_查询振幅、条件选股_查询流通市值、条件选股_查询每股收益ttm、条件选股_查询市盈率ttm、条件选股_查询净资产收益率、条件选股_查询每股净资产、条件选股_查询每股经营性现资金流、条件选股_查询毛利率、条件选股_查询净利率、条件选股_查询净利润、条件选股_查询净利润同比增长、条件选股_查询营业收入、条件选股_查询营收同比增长、条件选股_查询投资收入、条件选股_查询营业利润、条件选股_查询营业利润同比增长、条件选股_查询扣非净利润、条件选股_查询流动资产、条件选股_查询总资产、条件选股_查询短期负债、条件选股_查询总负债、条件选股_查询股东权益、条件选股_查询净经营性现金流、条件选股_查询净投资性现金流、条件选股_查询净融资性现金流、条件选股_查询流通股本、条件选股_查询总股本、条件选股_查询主力资金流入、条件选股_查询主力资金流出、条件选股_查询主力资金净流入、条件选股_查询总流入、条件选股_查询总流出、条件选股_查询高管名称、股票查询_查询代码、股票查询_查询开盘价、股票查询_查询最高价、股票查询_查询最低价、股票查询_查询当前价、股票查询_查询收盘价、股票查询_查询成交量、股票查询_查询成交额、股票查询_查询涨停价、股票查询_查询跌停价、股票查询_查询涨跌额、股票查询_查询涨跌幅、股票查询_查询换手率、股票查询_查询每股收益、股票查询_查询静态市盈率、股票查询_查询总市值、股票查询_查询振幅、股票查询_查询流通市值、股票查询_查询每股收益ttm、股票查询_查询市盈率ttm、股票查询_查询净资产收益率、股票查询_查询每股净资产、股票查询_查询每股经营性现金流、股票查询_查询毛利率、股票查询_查询净利率、股票查询_查询净利润、股票查询_查询净利润同比增长、股票查询_查询营业收入、股票查询_查询营收同比增长、股票查询_查询投资收入、股票查询_查询营业利润、股票查询_查询营业利润同比增长、股票查询_查询扣非净利润、股票查询_查询流动资产、股票查询_查询总资产、股票查询_查询短期负债、股票查询_查询总负债、股票查询_查询股东权益、股票查询_查询净经营性现金流、股票查询_查询净投资性现金流、股票查询_查询净融资性现金流、股票查询_查询银行资本充足率、股票查询_查询流通股本、股票查询_查询总股本、股票查询_查询主力资金流入、股票查询_查询主力资金流出、股票查询_查询主力资金净流入、股票查询_查询总流入、股票查询_查询总流出、股票查询_查询高管名称"
question = "请问我需要使用什么api来回答下面的问题? 请以(tool_name,api_name)格式返回所有需要的api，你只需要返回需要的api不需要调用。\n 以下是一个示例：(基金查询, 查询代码) (基金查询, 查询基金经理) (基金查询, 查询基金规模) (基金查询, 查询申购费率) \n我之前购买了浙商智选领航C，平均每天的收益率是多少，且对比同类基金，在1年的时间中所处的排名位次是多少？"
response = query_engine.query(question)
print(response)

为了回答您的问题，您需要使用(基金查询, 查询近期年化收益率) API。此API将返回您查询的基金在指定时间内的近期年化收益率以及它在同类基金中的排名情况。同时，您还需要使用(基金查询, 查询近期创新高次数) API来获取您查询的基金在指定时间范围内的近期创新高次数。这两个API将共同帮助您回答您的问题。
